In [1]:
import sys
import os
from dotenv import load_dotenv
load_dotenv()
os.chdir(os.getenv('PROJECT_DIRECTORY'))
sys.path.append(os.getenv('PROJECT_DIRECTORY'))

import pandas as pd
from datetime import datetime
import pytz
from src import supabaseUtils
from src import locationUtils

In [ ]:
suppliers = supabaseUtils.fetchFilteredRecords('companies', 'isSupplier', True)
standUpAnagrafica = pd.read_excel('data/anagrafica.xlsx')[['Ragione Sociale', 'city', 'Provincia', 'SEDE', 'Codice Cliente', 'FORNITORE']]
conversionSuppliers = {'Perletti': 'PERLETTI SPA',
                       'Vitakraft': 'VITAKRAFT ITALIA SPA',
                       'Sodifer': 'SODIFER SRL',
                       'Signify': 'SIGNIFY SPA',
                       'GoldPlast': 'GOLD PLAST SPA',
                       'MaFra': 'MAFRA SPA',
                       'BTicino': 'BTICINO SPA',
                       'Briloner': 'BRILONER ITALIA SRL',
                       'Colzani': 'COLZANI SPA',
                       'Incall': 'INCALL SPA',
                       'Mandelli': 'MANDELLI SRL'}
conversionSupplierCompanyId = dict(zip(suppliers['name'], suppliers['id']))

standUpAnagrafica['FORNITORE'] = standUpAnagrafica['FORNITORE'].map(conversionSuppliers)
standUpAnagrafica['supplierCompanyId'] = standUpAnagrafica['FORNITORE'].map(conversionSupplierCompanyId)
standUpAnagrafica = standUpAnagrafica.drop('FORNITORE', axis=1)
standUpAnagrafica = standUpAnagrafica.rename(columns={'Codice Cliente': 'sellerSellingPointCode'})

standUpAnagrafica

# sellingPoints

## addresses

In [ ]:
sellingPointsAddresses = standUpAnagrafica[['Ragione Sociale', 'city', 'Provincia']].drop_duplicates()
sellingPointsAddresses = sellingPointsAddresses.rename(columns={'Ragione Sociale': 'name',
                                                                'Provincia': 'province'})
# sellingPointsAddresses = locationUtils.processAddresses(sellingPointsAddresses)

# sellingPointsAddresses.to_excel('data/sellingPointsAddresses.xlsx', index=False)
sellingPointsAddresses.head(5)

In [ ]:
sellingPointsAddresses = pd.read_excel('data/sellingPointsAddresses.xlsx')
sellingPointsAddresses = pd.merge(sellingPointsAddresses.drop(columns=['city', 'province']), standUpAnagrafica[['Ragione Sociale', 'city', 'Provincia']].drop_duplicates().rename(columns={'Ragione Sociale': 'name', 'Provincia': 'stateProvince'}),
                                  on=['name'],
                                  how='left')

sellingPointsAddresses.drop(columns=['name']).to_csv('data/[UPLOAD] sellingPointsAddresses.csv', index=False)
sellingPointsAddresses.head(5)

## sellingPoints

In [ ]:
# Fetch all necessary data
sellers = supabaseUtils.fetchFilteredRecords('companies', 'isSeller', True)
allAddresses = supabaseUtils.fetchAllRecords('addresses')

# Create base selling points dataframe from anagrafica
sellingPoints = standUpAnagrafica[['Ragione Sociale', 'SEDE']].drop_duplicates().rename(columns={'Ragione Sociale': 'name'})

# Add seller company IDs to selling points
sellingPoints = pd.merge(sellingPoints, sellers[['id', 'name']].rename(columns={'id': 'sellerCompanyId', 'name': 'SEDE'}),
                         on=['SEDE'],
                         how='left').drop(columns=['SEDE'])

# Match addresses to selling points using coordinates
sellingPointsAddresses = pd.merge(sellingPointsAddresses[['name', 'latitude', 'longitude']], allAddresses[['id', 'latitude', 'longitude']].rename(columns={'id': 'addressId'}),
                                  on=['latitude', 'longitude'],
                                  how='left')

# Add address IDs to selling points
sellingPoints = pd.merge(sellingPoints, sellingPointsAddresses[['name', 'addressId']],
                         on=['name'],
                         how='left').drop_duplicates(subset=['name'], keep='first')

sellingPoints.to_csv('data/[UPLOAD] sellingPoints.csv', index=False)
sellingPoints.head(5)

# companySellingPoint

In [ ]:
sellingPoints = supabaseUtils.fetchAllRecords('sellingPoints').rename(columns={'id': 'sellingPointId'})
standUpAnagrafica = standUpAnagrafica.rename(columns={'Ragione Sociale': 'name'})

companySellingPoint = pd.merge(standUpAnagrafica[['name', 'supplierCompanyId', 'sellerSellingPointCode']], sellingPoints[['sellingPointId', 'name']],
                               on=['name'],
                               how='left')
companySellingPoint['startDate'] = datetime(2025, 1, 1, tzinfo=pytz.UTC)

companySellingPoint.drop(columns=['name']).to_csv('data/[UPLOAD] companySellingPoint.csv', index=False)
companySellingPoint.head(5)